Complete codes to processing CoastCams

# 📖 Table of Contents
- [Get camera intrinsics and extrinsics](#Get-camera-intrinsics-and-extrinsics)
- [Define timestacks for ARGUS camera](#Define-timestacks-for-ARGUS-camera)
- [Rectify image products](#Rectify-image-products)
    - [Example: Plot image rectification from netcdf](##Example-plot-of-rectified-image)
    - [Example: Rectify single image](##Rectifying-single-image)
- [Extract runup](#Extract-runup)
- [Extract shorelines](#Extract-shorelines)


## Get camera intrinsics and extrinsics

## Define timestacks for ARGUS camera

In [ ]:
import sys
import os
import json
if 'CODES' not in sys.path:
    sys.path.append('CODES')
    
from utils_runup import *
from ImageHandler import ImageHandler

configPath = "config.json"
sample_img_path = "DATA/DATA/images/1737649800.Thu.Jan.23_16_30_00.GMT.2025.CACO03.c2.snap.jpg"

#yaml_filename = "filename dictionary of coordinates E,N,Z,U,V - end in .yaml"
yaml_filename = 'example.yaml'
#pix_filename = "filename for .pix file - end in .pix"
pix_filename = 'example.pix'

img_handle = ImageHandler(imagePath = sample_img_path, configPath = configPath)

img_handle.read_metadata()
metadata = img_handle.metadata
config = img_handle.config

productsPath = config.get("productsPath", {})
pixsaveDir = config.get("pixsaveDir", {})

define_UV_transects(productsPath, 
                    metadata['intrinsics'], 
                    metadata['extrinsics'], 
                    pixsaveDir=pixsaveDir, 
                    yaml_filename = yaml_filename, 
                    pix_filename = pix_filename)         



## Rectify image products

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore

datastore = ImageDatastore(configPath = 'config.json')
datastore.image_stats()
datastore.keep_images_by_type(['snap', 'timex', 'var', 'bright', 'dark'])
datastore.keep_images_by_type(['timex'])
#datastore.initialize_image_handlers()
datastore.merge_images()

#datastore.apply_to_all_images('rectify_image')
datastore.apply_to_all_images('save_to_netcdf')

datastore.create_video(image_type = 'timex', frame_rate = 2, camera = '2')#, end_time=datetime(2024, 8,17,00,00,00, tzinfo=timezone.utc))


### Example plot of rectified image

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
ds = xr.open_dataset("DATA/DATA/results/netcdf/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.timex.nc")
ds

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle(f"Rectified Image - {ds.attrs['name']}", fontsize=14)

# First subplot: pcolor
ax1 = axes[0]
pcolor_plot = ax1.pcolor(ds['Eastings'], ds['Northings'], ds['Ir'], shading='auto')
ax1.scatter(ds.attrs['origin_easting'], ds.attrs['origin_northing'], c='r')
ax1.set_xlabel("Eastings")
ax1.set_ylabel("Northings")

# Second subplot: imshow
ax2 = axes[1]
im = ax2.imshow(
    ds['Ir'],
    extent=[ds['localX'][0, 0], ds['localX'][-1, -1], ds['localY'][0, 0], ds['localY'][-1, -1]],
    origin='lower'
)
ax2.set_xlabel("Local X")
ax2.set_ylabel("Local Y")

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for title


### Rectifying single image

In [ ]:
# rectify single image
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageHandler

imagePath = 'path/to/image'
image_handler = ImageHandler(imagePath = imagePath, configPath = 'config.json')
image_handler.rectify_image()
image_handler.save_to_netcdf()

## Extract runup 

In [2]:
import sys
import json
if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore
from utils_runup import *
with open('config.json', "r") as f:
    config = json.load(f)

#split_tiff(config)

datastore = ImageDatastore(configPath = config)
datastore.remove_images_by_type(['snap', 'timex', 'var', 'bright', 'dark'])
datastore.keep_images_by_type(['ras_transect4'])
datastore.image_stats()

datastore.get_runup_from_timestacks(segformer_flag=False, save_flag=False)


Total images: 5
Image types and their counts:
  ras_transect4: 5
Processing: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/1730026800.Sun.Oct.27_11_00_00.GMT.2024.CACO03.c1.ras_transect4.png.
Extracting runup
Saved overlay image: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/results/runup/overlay_1730026800.Sun.Oct.27_11_00_00.GMT.2024.CACO03.c1.ras_transect4.jpg
Processing: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/1730028600.Sun.Oct.27_11_30_00.GMT.2024.CACO03.c1.ras_transect4.png.
Extracting runup
Saved overlay image: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/results/runup/overlay_1730028600.Sun.Oct.27_11_30_00.GMT.2024.CACO03.c1.ras_transect4.jpg
Processing: /mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/1730048400.Sun.Oct.27_17_00_00.GMT

## Extract shorelines

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')

from ImageHandler import ImageDatastore

datastore = ImageDatastore(configPath = 'config.json')
datastore.keep_images_by_type([ 'bright'])
datastore.image_stats()

datastore.process_image_pairs(make_plots=False)
datastore.apply_to_all_images('rectify_image')
datastore.apply_to_all_images('save_to_netcdf')

#### For individual image

In [ ]:
import sys

if 'CODES' not in sys.path:
    sys.path.append('CODES')
from ImageHandler import ImageHandler
import utils_CIRN
import json
import numpy as np
img_handle = ImageHandler(configPath = 'config.json', imagePath = '/mnt/c/Users/alange/Desktop/DATA/CODES/GitHub/ShoreScan/ShoreScan_clean_example/DATA/DATA/images/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.bright.jpg')
img_handle.rectify_image()

img_handle.process_bright()
img_handle.save_to_netcdf()

#### For plotting

In [ ]:
ds = xr.open_dataset('DATA/DATA/results/netcdf/1729620000.Tue.Oct.22_18_00_00.GMT.2024.CACO03.c1.bright-shoreline-rectify.nc')

In [ ]:
import plotly.express as px
px.set_mapbox_access_token('open-street-map')

In [ ]:
import numpy as np
import pyproj
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.feature import LAND, OCEAN
import utm

# Define the map with the LCC (Lambert Conformal Conic) projection
lat_0, lon_0 = utm.to_latlon(ab['rectified_image']['Origin_Easting'], 
                              ab['rectified_image']['Origin_Northing'], 
                              ab['rectified_image']['Origin_UTMZone'], 'N')

# Correcting the latitude and longitude bounds (fixing the mistake in your original code)
lat_min, lat_max = lat_0 - 0.01, lat_0 + 0.01  # Example latitude bounds
lon_min, lon_max = lon_0 - 0.01, lon_0 + 0.01  # Example longitude bounds

# Define the UTM projection for Zone 19N (EPSG:32619) and WGS84 (EPSG:4326)
utm_proj = pyproj.CRS("EPSG:32619")  # UTM Zone 19N
wgs84 = pyproj.CRS("EPSG:4326")  # WGS84 Latitude/Longitude

# Create a transformer object to convert from UTM to WGS84
transformer = pyproj.Transformer.from_crs(utm_proj, wgs84, always_xy=True)

# Sample 2D array of Eastings (x) and Northings (y) in UTM (replace with your data)
eastings = np.array(ab['rectified_image']['Eastings'])
northings = np.array(ab['rectified_image']['Northings'])

# Convert UTM (Eastings/Northings) to Latitude/Longitude
longitudes,latitudes = transformer.transform(eastings, northings)


In [ ]:

# Define the map with a PlateCarree projection (Lat/Lon)
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())  # Adjust extents to your data


# Add map features
ax.add_feature(LAND)
ax.add_feature(OCEAN, edgecolor='blue')
# Create a Cartopy projection (using LambertConformal projection as in the Basemap example)
projection = ccrs.LambertConformal(central_longitude=lon_0, central_latitude=lat_0)
#ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())  # Set extent based on min/max


ax.scatter(lon_0, lat_0, color='red', transform=ccrs.PlateCarree())  # Scatter on the map
projection = ccrs.LambertConformal(central_longitude=lon_0, central_latitude=lat_0)

# Create a transformation from Lat/Lon to the map projection (Lambert Conformal)
#latitudes_flat = latitudes.flatten()
#longitudes_flat = longitudes.flatten()

# Create a transformation from Lat/Lon to the map projection (Lambert Conformal)
#x_flat, y_flat = projection.transform_points(ccrs.PlateCarree(), longitudes_flat, latitudes_flat)[:, :2].T

# Reshape the transformed points back into 2D arrays for plotting
#x = x_flat.reshape(latitudes.shape)
#y = y_flat.reshape(latitudes.shape)

# Plot the data on the map
plt.pcolor(x,y, ab['rectified_image']['Ir'])#, cmap='viridis', shading='auto')

gl = ax.gridlines(draw_labels=True)  # draw_labels=True adds labels to the gridlines
gl.xlabels_top = False  # Don't show longitude labels on top
gl.ylabels_right = False  # Don't show latitude labels on the right

# Show the map
plt.show()
print('hello')


In [ ]:
# Create a Cartopy projection (using LambertConformal projection as in the Basemap example)
projection = ccrs.LambertConformal(central_longitude=lon_0, central_latitude=lat_0)

# Create a transformation from Lat/Lon to the map projection (Lambert Conformal)
latitudes_flat = latitudes.flatten()
longitudes_flat = longitudes.flatten()

# Create a transformation from Lat/Lon to the map projection (Lambert Conformal)
x_flat, y_flat = projection.transform_points(ccrs.PlateCarree(), longitudes_flat, latitudes_flat)[:, :2].T

# Reshape the transformed points back into 2D arrays for plotting
x = x_flat.reshape(latitudes.shape)
y = y_flat.reshape(latitudes.shape)

# Plot the data on the map
plt.pcolor(x,y, ab['rectified_image']['Ir'])#, cmap='viridis', shading='auto')


In [ ]:
import matplotlib.pyplot as plt
plt.pcolor(latitudes, longitudes, ab['rectified_image']['Ir'])
